In [2]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch, BayesianOptimization
import tensorflow as tf

In [3]:
df_train = pd.read_csv('/home/ubuntu/cs230-project/data_sub_PCA_mild/df_train_PCA_dict_df_metab.csv')
print (df_train.head)

<bound method NDFrame.head of      Depressed at Time  df_metab_0  df_metab_1  df_metab_2  df_metab_3  \
0                 True   45.789637   14.994401    2.792549   -7.301174   
1                False   21.305736  -12.739211    2.414824  -22.494078   
2                False  -11.160583    0.500590    1.948961  -18.053438   
3                 True   -3.505071    3.023377  -14.520562    9.177950   
4                False    2.370956   14.234072    6.691360    4.755687   
..                 ...         ...         ...         ...         ...   
123              False    0.125152   -2.485553   -3.342023  -20.668625   
124              False  -16.114327  -22.152604  -23.709256  -18.545960   
125              False   36.381486    6.960738   11.266882    9.134153   
126              False   -2.210158   12.276662  -22.779149   -9.248137   
127              False   -1.710068   17.256411   11.794209    3.967151   

     df_metab_4  df_metab_5  df_metab_6  df_metab_7  df_metab_8  ...  \
0      2.

In [4]:
y_train = df_train.iloc[:,0]
X_train = df_train.iloc[:,1:]

In [5]:
# one-hot encoding
y_train = pd.get_dummies(y_train)[True]
print (y_train.sum())

25


In [6]:
print (y_train.shape)
print (X_train.shape)

(128,)
(128, 88)


In [7]:
df_dev = pd.read_csv('/home/ubuntu/cs230-project/data_sub_PCA_mild/df_dev_PCA_dict_df_metab.csv')
print (df_dev.shape)

(37, 89)


In [8]:
y_dev = df_dev.iloc[:,0]
y_dev = pd.get_dummies(y_dev)[True]
X_dev = df_dev.iloc[:,1:]

In [9]:
print (y_dev.shape)
print (X_dev.shape)
print (y_dev.sum())

(37,)
(37, 88)
5


In [10]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(units=70, activation='relu',
                          kernel_regularizer = keras.regularizers.l2(hp.Float('l2_dns1',0, 0.02, step=0.001))))
    model.add(layers.Dropout(hp.Float('dropout1', 0, 0.5, step = 0.1)))
    model.add(layers.Dense(units=20, activation='relu',
                          kernel_regularizer = keras.regularizers.l2(hp.Float('l2_dns2',0, 0.02, step=0.001))))
    model.add(layers.Dropout(hp.Float('dropout2', 0, 0.5, step = 0.1)))
    #model.add(layers.Dense(2, activation = 'softmax')) #2 for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))
    
    metrics_list = [
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ]
    
    model.compile(
        optimizer = keras.optimizers.RMSprop(
        learning_rate = 0.001,
        rho = 0.75,
        momentum = 0.15),
        loss = 'binary_crossentropy',
        metrics = metrics_list)
    
    return model

In [11]:
#tuner = RandomSearch(
tuner = BayesianOptimization(
    build_model,
    objective = 'val_binary_accuracy',
    max_trials = 5,
    executions_per_trial = 3,
    directory = 'Metabolomics_dev_mild')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [12]:
tuner.search(X_train.values, y_train.values,
            epochs=5,
            validation_data=(X_dev, y_dev))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 128 samples, validate on 37 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 2.9891 - binary_accuracy: 0.5625 - precision: 0.2000 - recall: 0.60 - 1s 4ms/sample - loss: 2.7386 - binary_accuracy: 0.6562 - precision: 0.2432 - recall: 0.3600 - val_loss: 6.4587 - val_binary_accuracy: 0.7027 - val_precision: 0.2000 - val_recall: 0.4000
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 1.6467 - binary_accuracy: 0.8125 - precision: 0.3333 - recall: 0.20 - 0s 262us/sample - loss: 2.3732 - binary_accuracy: 0.6875 - precision: 0.2414 - recall: 0.2800 - val_loss: 6.1494 - val_binary_accuracy: 0.7297 - val_precision: 0.2222 - val_recall: 0.4000
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 2.2488 - binary_accuracy: 0.7500 - precision: 0.4444 - recall: 0.57 - 0s 249us/sample - loss: 2.1868 - binary_accuracy: 0.7422 - precision: 

Train on 128 samples, validate on 37 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 15.2230 - binary_accuracy: 0.2188 - precision: 0.1724 - recall: 0.833 - 0s 3ms/sample - loss: 12.7256 - binary_accuracy: 0.2344 - precision: 0.1983 - recall: 0.9600 - val_loss: 20.0772 - val_binary_accuracy: 0.1351 - val_precision: 0.0909 - val_recall: 0.6000
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 11.0005 - binary_accuracy: 0.3438 - precision: 0.3000 - recall: 1.000 - 0s 237us/sample - loss: 9.0722 - binary_accuracy: 0.3125 - precision: 0.2162 - recall: 0.9600 - val_loss: 15.9051 - val_binary_accuracy: 0.1892 - val_precision: 0.0690 - val_recall: 0.4000
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 7.0067 - binary_accuracy: 0.5000 - precision: 0.3600 - recall: 1.00 - 0s 214us/sample - loss: 6.8028 - binary_accuracy: 0.3438 - precision: 0.2136 - recall: 0.8800 - val_loss: 13.1023 - val_binary_accuracy: 0.2162 - val_precisi

Train on 128 samples, validate on 37 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 4.6655 - binary_accuracy: 0.3750 - precision: 0.0588 - recall: 0.20 - 0s 3ms/sample - loss: 3.6386 - binary_accuracy: 0.4766 - precision: 0.0962 - recall: 0.2000 - val_loss: 6.0845 - val_binary_accuracy: 0.7838 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 3.1338 - binary_accuracy: 0.5938 - precision: 0.2500 - recall: 0.22 - 0s 229us/sample - loss: 3.1758 - binary_accuracy: 0.6172 - precision: 0.1667 - recall: 0.2400 - val_loss: 6.3712 - val_binary_accuracy: 0.8108 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 1.8446 - binary_accuracy: 0.7188 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 180us/sample - loss: 2.7513 - binary_accuracy: 0.6406 - precision: 0.1111 - recall: 0.1200 - val_loss: 5.9000 - val_binary_accuracy: 0.81

Train on 128 samples, validate on 37 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 4.4569 - binary_accuracy: 0.5625 - precision: 0.2857 - recall: 0.50 - 0s 3ms/sample - loss: 3.6085 - binary_accuracy: 0.6484 - precision: 0.2619 - recall: 0.4400 - val_loss: 7.4852 - val_binary_accuracy: 0.7297 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 2.7409 - binary_accuracy: 0.7188 - precision: 0.2000 - recall: 0.16 - 0s 223us/sample - loss: 2.7076 - binary_accuracy: 0.7266 - precision: 0.2727 - recall: 0.2400 - val_loss: 7.4134 - val_binary_accuracy: 0.7568 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 2.2504 - binary_accuracy: 0.8438 - precision: 0.6667 - recall: 0.33 - 0s 176us/sample - loss: 2.2806 - binary_accuracy: 0.8125 - precision: 0.5238 - recall: 0.4400 - val_loss: 7.1501 - val_binary_accuracy: 0.7568 - val

Train on 128 samples, validate on 37 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 8.5941 - binary_accuracy: 0.2188 - precision: 0.1538 - recall: 0.57 - 0s 3ms/sample - loss: 8.3330 - binary_accuracy: 0.2891 - precision: 0.1827 - recall: 0.7600 - val_loss: 17.5205 - val_binary_accuracy: 0.3243 - val_precision: 0.0833 - val_recall: 0.4000
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 5.1616 - binary_accuracy: 0.4688 - precision: 0.2000 - recall: 0.80 - 0s 230us/sample - loss: 5.4334 - binary_accuracy: 0.4375 - precision: 0.1948 - recall: 0.6000 - val_loss: 16.7228 - val_binary_accuracy: 0.5405 - val_precision: 0.0714 - val_recall: 0.2000
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 5.3389 - binary_accuracy: 0.5312 - precision: 0.2143 - recall: 0.42 - 0s 217us/sample - loss: 4.3665 - binary_accuracy: 0.5469 - precision: 0.2203 - recall: 0.5200 - val_loss: 15.3677 - val_binary_accuracy: 0.6216 - val_precision: 0

INFO:tensorflow:Oracle triggered exit


In [13]:
tuner.results_summary()

In [14]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print (best_hp.values)

{'l2_dns1': 0.013000000000000001, 'dropout1': 0.2, 'l2_dns2': 0.003, 'dropout2': 0.1}


In [18]:
best_model = tuner.get_best_models(num_models=1)[0]

In [19]:
best_model.fit(x=X_train.values, y=y_train.values, epochs=20)

Train on 128 samples
Epoch 1/20
128/128 [==============================] - ETA: 0s - loss: 1.9671 - binary_accuracy: 0.8125 - precision: 0.4000 - recall: 0.40 - 0s 1ms/sample - loss: 2.0130 - binary_accuracy: 0.7500 - precision: 0.3478 - recall: 0.3200
Epoch 2/20
128/128 [==============================] - ETA: 0s - loss: 2.0964 - binary_accuracy: 0.7500 - precision: 0.4545 - recall: 0.71 - 0s 129us/sample - loss: 1.8729 - binary_accuracy: 0.7422 - precision: 0.3462 - recall: 0.3600
Epoch 3/20
128/128 [==============================] - ETA: 0s - loss: 2.3562 - binary_accuracy: 0.7188 - precision: 0.3333 - recall: 0.12 - 0s 122us/sample - loss: 1.8930 - binary_accuracy: 0.7578 - precision: 0.3636 - recall: 0.3200
Epoch 4/20
128/128 [==============================] - ETA: 0s - loss: 1.5889 - binary_accuracy: 0.7812 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 119us/sample - loss: 1.9784 - binary_accuracy: 0.7734 - precision: 0.4000 - recall: 0.3200
Epoch 5/20
128/128 [=================

In [20]:
results = best_model.evaluate(X_dev.values, y_dev.values)

37/37 [==============================] - ETA: 0s - loss: 3.7771 - binary_accuracy: 0.8438 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 2ms/sample - loss: 3.3952 - binary_accuracy: 0.8649 - precision: 0.0000e+00 - recall: 0.0000e+00


In [21]:
print ("test loss, test acc:", results)

test loss, test acc: [3.39521458181175, 0.8648649, 0.0, 0.0]


In [22]:
print (pd.DataFrame({'ground_truth': y_dev,
                     'prediction' : best_model.predict(X_dev.values)[:,0]>0.5}))

    ground_truth  prediction
0              0       False
1              0       False
2              0       False
3              1       False
4              1       False
5              1       False
6              1       False
7              0       False
8              0       False
9              0       False
10             0       False
11             0       False
12             0       False
13             1       False
14             0       False
15             0       False
16             0       False
17             0       False
18             0       False
19             0       False
20             0       False
21             0       False
22             0       False
23             0       False
24             0       False
25             0       False
26             0       False
27             0       False
28             0       False
29             0       False
30             0       False
31             0       False
32             0       False
33            